### Scraping

In [ ]:
def importxml(page, xpath):
    tree = html.fromstring(page.content)
    content_list = tree.xpath(xpath)
    content = content_list[0].text_content() if len(content_list) > 0 else ''
    default_encoding = sys.getdefaultencoding()
    print(f"content.strip(): {content.strip()}")
    return content.strip()

### Retirada das informações da FINEP

In [ ]:
import requests
from lxml import html
import pandas as pd

# Função para extrair o conteúdo com base no XPath
def importxml(page, xpath):
    tree = html.fromstring(page.content)
    content_list = tree.xpath(xpath)
    content = content_list[0].text_content() if len(content_list) > 0 else ''
    return content.strip()

# Função para extrair todos os links de PDF de um edital
def extract_pdfs_from_edital(url, edital_number):
    page = requests.get(url)

    # XPath para encontrar todas as linhas que possuem documentos
    xpath_base = f'//*[@id="documentos-item-{edital_number}"]/tbody/tr/td[3]/a/@href'

    # Extrair todos os links para PDFs
    pdf_links = html.fromstring(page.content).xpath(xpath_base)

    return pdf_links

# Lista com os números de editais que você deseja percorrer
editais_numbers = list(range(586, 748))  # Exemplo de 586 até 747

# Lista para armazenar as informações
data = []

# Loop por cada edital
for edital_number in editais_numbers:
    url = f'http://finep.gov.br/chamadas-publicas/chamadapublica/{edital_number}'
    pdf_links = extract_pdfs_from_edital(url, edital_number)

    # Adiciona cada PDF do edital à lista de dados
    for pdf_link in pdf_links:
        data.append({'edital_number': edital_number, 'pdf_link': pdf_link})

# Convertendo a lista em dataframe
df = pd.DataFrame(data)

# Salvar os dados em um arquivo CSV
# df.to_csv('editais_finep.csv', index=False)

print("Extração concluída e dados salvos em 'editais_finep.csv'")


Extração concluída e dados salvos em 'editais_finep.csv'


In [ ]:
df

,edital_number,pdf_link
0,586,http://download.finep.gov.br/chamadas/cooperac...
1,586,http://download.finep.gov.br/chamadas/cooperac...
2,586,http://download.finep.gov.br/chamadas/cooperac...
3,586,http://download.finep.gov.br/chamadas/cooperac...
4,587,/images/chamadas-publicas/2015/Edital_de_Subve...
...,...,...
2005,746,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...
2006,746,/images/chamadas-publicas/2024/22_08_2024_Form...
2007,746,https://forms.finep.gov.br/forms/
2008,747,/images/chamadas-publicas/2024/01_10_2024_Cart...


In [ ]:
def fix_link(link):
    if link.startswith('/'):
        return "http://finep.gov.br" + link
    return link

df["link"] = df["pdf_link"].apply(fix_link)
df

,edital_number,pdf_link,link
0,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
1,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
2,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
3,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
4,587,/images/chamadas-publicas/2015/Edital_de_Subve...,http://finep.gov.br/images/chamadas-publicas/2...
...,...,...,...
2005,746,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...
2006,746,/images/chamadas-publicas/2024/22_08_2024_Form...,http://finep.gov.br/images/chamadas-publicas/2...
2007,746,https://forms.finep.gov.br/forms/,https://forms.finep.gov.br/forms/
2008,747,/images/chamadas-publicas/2024/01_10_2024_Cart...,http://finep.gov.br/images/chamadas-publicas/2...


In [ ]:
df.to_csv('editais_finep.csv', index=False)

### Extração dos dados dos pdfs

In [ ]:
!pip install pdfplumber
!pip install pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import os
import pdfplumber
from PyPDF2 import PdfReader
from requests.exceptions import ConnectionError, Timeout, RequestException

# Função para baixar o PDF a partir de um link
# def download_pdf(pdf_url, output_path):
#     response = requests.get(pdf_url)
#     with open(output_path, 'wb') as file:
#         file.write(response.content)

def download_pdf(pdf_url, output_path):
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()  # Verifica se há algum erro HTTP
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"PDF baixado com sucesso: {pdf_url}")
    except ConnectionError:
        print(f"Erro de conexão ao tentar acessar {pdf_url}")
    except Timeout:
        print(f"O pedido para {pdf_url} expirou")
    except RequestException as e:
        print(f"Um erro ocorreu ao tentar baixar {pdf_url}: {e}")

# Função para extrair texto de um PDF
# def extract_text_from_pdf(pdf_path):
#     text = ''
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             text += page.extract_text() + '\n'
#     return text

def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text

# Carregar o dataframe com os links dos PDFs
df = pd.read_csv('editais_finep.csv')

# Diretório para salvar os PDFs baixados
pdf_dir = 'pdfs_finep'
os.makedirs(pdf_dir, exist_ok=True)

# Lista para armazenar os textos extraídos dos PDFs
pdf_texts = []

# Loop sobre os links no dataframe
for index, row in df.iterrows():
    pdf_link = row['link']
    edital_number = row['edital_number']

    # Nome do arquivo PDF a ser salvo
    pdf_filename = f'edital_{edital_number}_{index}.pdf'
    pdf_path = os.path.join(pdf_dir, pdf_filename)

    # Baixar o PDF
    print(f"Baixando {pdf_link}...")
    download_pdf(pdf_link, pdf_path)

    # Extrair o texto do PDF
    print(f"Extraindo texto de {pdf_filename}...")
    pdf_text = extract_text_from_pdf(pdf_path)

    # Armazenar no dataframe
    pdf_texts.append({'edital_number': edital_number, 'pdf_link': pdf_link, 'pdf_text': pdf_text})

# Converter a lista em um dataframe
df_texts = pd.DataFrame(pdf_texts)

# Salvar o dataframe com o texto dos PDFs
df_texts.to_csv('editais_finep_textos.csv', index=False)

print("Extração de textos concluída e dados salvos em 'editais_finep_textos.csv'")

Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaport..pdf...
PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaport..pdf
Extraindo texto de edital_586_0.pdf...
Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaingles.pdf...


PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaingles.pdf
Extraindo texto de edital_586_1.pdf...
Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoport..pdf...
PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoport..pdf
Extraindo texto de edital_586_2.pdf...
Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoingles.pdf...


PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoingles.pdf
Extraindo texto de edital_586_3.pdf...
Baixando http://finep.gov.br/images/chamadas-publicas/2015/Edital_de_Subvencao_Economica_a_Inovacao-Transferencia_de_Tecnologia_do_SGDC-AEB_e_Finep.pdf...
PDF baixado com sucesso: http://finep.gov.br/images/chamadas-publicas/2015/Edital_de_Subvencao_Economica_a_Inovacao-Transferencia_de_Tecnologia_do_SGDC-AEB_e_Finep.pdf
Extraindo texto de edital_587_4.pdf...
Baixando http://finep.gov.br/images/chamadas-publicas/2015/Resultado_Final_SGDC.pdf...
PDF baixado com sucesso: http://finep.gov.br/images/chamadas-publicas/2015/Resultado_Final_SGDC.pdf
Extraindo texto de edital_587_5.pdf...
Baixando http://finep.gov.br/images/chamadas-publicas/2015/Resultado-Preliminar-Edital_SBV_TT_SGDC_01-2015.pdf...
PDF baixado com sucesso: http://finep.gov.br/images/chamadas-publicas/2015/Resultado-Preliminar-Edital_SBV

FileNotFoundError: [Errno 2] No such file or directory: 'pdfs_finep/edital_587_9.pdf'

In [ ]:
df

,edital_number,pdf_link,link
0,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
1,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
2,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
3,586,http://download.finep.gov.br/chamadas/cooperac...,http://download.finep.gov.br/chamadas/cooperac...
4,587,/images/chamadas-publicas/2015/Edital_de_Subve...,http://finep.gov.br/images/chamadas-publicas/2...
...,...,...,...
2005,746,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...
2006,746,/images/chamadas-publicas/2024/22_08_2024_Form...,http://finep.gov.br/images/chamadas-publicas/2...
2007,746,https://forms.finep.gov.br/forms/,https://forms.finep.gov.br/forms/
2008,747,/images/chamadas-publicas/2024/01_10_2024_Cart...,http://finep.gov.br/images/chamadas-publicas/2...


In [ ]:
import requests
import pandas as pd
import os
from PyPDF2 import PdfReader
from requests.exceptions import ConnectionError, Timeout, RequestException

# Função para baixar o PDF a partir de um link
def download_pdf(pdf_url, output_path):
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()  # Verifica se há algum erro HTTP
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"PDF baixado com sucesso: {pdf_url}")
        return True  # Retorna True se o download foi bem-sucedido
    except ConnectionError:
        print(f"Erro de conexão ao tentar acessar {pdf_url}")
    except Timeout:
        print(f"O pedido para {pdf_url} expirou")
    except RequestException as e:
        print(f"Um erro ocorreu ao tentar baixar {pdf_url}: {e}")
    return False  # Retorna False se o download falhou

# Função para extrair texto de um PDF
def extract_text_from_pdf(pdf_path):
    text = ''
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + '\n'
    except Exception as e:
        print(f"Erro ao extrair texto de {pdf_path}: {e}")
    return text

# Carregar o dataframe com os links dos PDFs
df = pd.read_csv('editais_finep.csv')

# Diretório para salvar os PDFs baixados
pdf_dir = 'pdfs_finep'
os.makedirs(pdf_dir, exist_ok=True)

# Lista para armazenar os textos extraídos dos PDFs
pdf_texts = []

# Loop sobre os links no dataframe
for index, row in df.iterrows():
    pdf_link = row['link']
    edital_number = row['edital_number']

    # Nome do arquivo PDF a ser salvo
    pdf_filename = f'edital_{edital_number}_{index}.pdf'
    pdf_path = os.path.join(pdf_dir, pdf_filename)

    # Baixar o PDF
    print(f"Baixando {pdf_link}...")
    if download_pdf(pdf_link, pdf_path):
        # Extrair o texto do PDF somente se o download foi bem-sucedido
        print(f"Extraindo texto de {pdf_filename}...")
        pdf_text = extract_text_from_pdf(pdf_path)
    else:
        pdf_text = "Erro no download, não foi possível extrair o texto."

    # Armazenar no dataframe
    pdf_texts.append({'edital_number': edital_number, 'pdf_link': pdf_link, 'pdf_text': pdf_text})

# Converter a lista em um dataframe
df_texts = pd.DataFrame(pdf_texts)

# Salvar o dataframe com o texto dos PDFs
df_texts.to_csv('editais_finep_textos.csv', index=False)

print("Extração de textos concluída e dados salvos em 'editais_finep_textos.csv'")


Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaport..pdf...
PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaport..pdf
Extraindo texto de edital_586_0.pdf...
Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaingles.pdf...


PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/editais/20141114Finep-Tekes-ChamadaConjuntaingles.pdf
Extraindo texto de edital_586_1.pdf...
Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoport..pdf...
PDF baixado com sucesso: http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoport..pdf
Extraindo texto de edital_586_2.pdf...
Baixando http://download.finep.gov.br/chamadas/cooperacao_internacional/documentos/20141114Finep-Tekes-FormulariodeSolicitacaoingles.pdf...


Streaming output truncated to the last 5000 lines.
Baixando http://finep.gov.br/images/chamadas-publicas/2020/23_07_2020__Resultado_Preliminar_2_Fase_Mulheres_Inovadoras.pdf...
PDF baixado com sucesso: http://finep.gov.br/images/chamadas-publicas/2020/23_07_2020__Resultado_Preliminar_2_Fase_Mulheres_Inovadoras.pdf
Extraindo texto de edital_638_393.pdf...
Baixando http://finep.gov.br/images/chamadas-publicas/2020/22_05_2020_Edital_Mulheres_Inovadoras_v_21_05_2020.pdf...
PDF baixado com sucesso: http://finep.gov.br/images/chamadas-publicas/2020/22_05_2020_Edital_Mulheres_Inovadoras_v_21_05_2020.pdf
Extraindo texto de edital_638_394.pdf...
Baixando http://finep.gov.br/images/chamadas-publicas/2020/15_05_2020_Mulheres_Inovadoras_resultado_final_1_fase.pdf...
PDF baixado com sucesso: http://finep.gov.br/images/chamadas-publicas/2020/15_05_2020_Mulheres_Inovadoras_resultado_final_1_fase.pdf
Extraindo texto de edital_638_395.pdf...
Baixando http://finep.gov.br/images/chamadas-publicas/2020/17

In [ ]:
df_texts

,edital_number,pdf_link,pdf_text
0,586,http://download.finep.gov.br/chamadas/cooperac...,\n \n \n 1 www.finep.gov.br \n \n \nRio d...
1,586,http://download.finep.gov.br/chamadas/cooperac...,\n \n \n 1 www.finep.gov.br \n \n \nRio d...
2,586,http://download.finep.gov.br/chamadas/cooperac...,\n 1 www.finep.gov.br \n \n \nRio de Janeir...
3,586,http://download.finep.gov.br/chamadas/cooperac...,\n \n \n 1 www.finep.gov.br \n \n \nRio d...
4,587,http://finep.gov.br/images/chamadas-publicas/2...,\n1 \n \n \nEDITAL DE SELEÇÃO...
...,...,...,...
2005,746,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...,
2006,746,http://finep.gov.br/images/chamadas-publicas/2...,
2007,746,https://forms.finep.gov.br/forms/,
2008,747,http://finep.gov.br/images/chamadas-publicas/2...,\n \n \n \n \nCARTA CONVITE MCTI/FINEP – PRO...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


### Criação do RAG

#### Embedding

In [ ]:
!pip install -q transformers sentence_transformers faiss-cpu torch nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 49.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import os
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from google.colab import userdata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
HUGGING_FACE_ACCESS_TOKEN = userdata.get('HUGGING_FACE_ACCESS_TOKEN')

model_name = 'google/gemma-2-2b-it'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    token=HUGGING_FACE_ACCESS_TOKEN
    ).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name, token=HUGGING_FACE_ACCESS_TOKEN)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def split_text_into_chunks(text, max_chunk_size=1000):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

def text_to_embeddings(text):
    if isinstance(text, str):
        chunks = split_text_into_chunks(text)
        document_embeddings = encoder.encode(chunks)
        return document_embeddings
    return np.array([])

def get_chunks(text):
  if isinstance(text, str):
    chunks = split_text_into_chunks(text)
    return chunks
  return []

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df_texts = pd.DataFrame()
try:
  df_texts = pd.read_csv('editais_finep_textos.csv')
except:
  df_texts = df_texts
  print("Arquivo não encontrado, utilizando dataframe em memória")
df_rag = df_texts.copy()
df_rag['embeddings'] = df_rag['pdf_text'].apply(text_to_embeddings)
df_rag['text_chunks'] = df_rag['pdf_text'].apply(get_chunks)

#### Vector Store

In [ ]:
df_rag

,edital_number,pdf_link,pdf_text,embeddings,text_chunks
0,586,http://download.finep.gov.br/chamadas/cooperac...,\n \n \n 1 www.finep.gov.br \n \n \nRio d...,"[[-0.058180064, -0.0136899445, 0.013687223, -0...",[1 www.finep.gov.br \n \n \nRio de Janeir o \...
1,586,http://download.finep.gov.br/chamadas/cooperac...,\n \n \n 1 www.finep.gov.br \n \n \nRio d...,"[[-0.09705758, -0.060008448, 0.014778403, -0.0...",[1 www.finep.gov.br \n \n \nRio de Janeiro \...
2,586,http://download.finep.gov.br/chamadas/cooperac...,\n 1 www.finep.gov.br \n \n \nRio de Janeir...,"[[-0.017535595, -0.002232036, -0.016732598, -0...",[1 www.finep.gov.br \n \n \nRio de Janeiro \...
3,586,http://download.finep.gov.br/chamadas/cooperac...,\n \n \n 1 www.finep.gov.br \n \n \nRio d...,"[[-0.017170914, -0.0019149019, -0.016095396, -...",[1 www.finep.gov.br \n \n \nRio de Janeiro \...
4,587,http://finep.gov.br/images/chamadas-publicas/2...,\n1 \n \n \nEDITAL DE SELEÇÃO...,"[[0.036442973, -0.06687919, -0.06770537, -0.03...",[1 \n \n \nEDITAL DE SELEÇÃO PÚBLICA MCTI/AEB/...
...,...,...,...,...,...
2005,746,https://docs.google.com/forms/d/e/1FAIpQLSc8aY...,NaN,[],[]
2006,746,http://finep.gov.br/images/chamadas-publicas/2...,NaN,[],[]
2007,746,https://forms.finep.gov.br/forms/,NaN,[],[]
2008,747,http://finep.gov.br/images/chamadas-publicas/2...,\n \n \n \n \nCARTA CONVITE MCTI/FINEP – PRO...,"[[0.034669165, -0.007871994, -0.025804847, -0....",[CARTA CONVITE MCTI/FINEP – PROGRAMA NACIONAL ...


In [ ]:
# all_embeddings = np.vstack(df_rag['embeddings'].tolist())
valid_embeddings = df_rag['embeddings'].apply(lambda x: len(x) > 0)
filtered_embeddings = df_rag[valid_embeddings]['embeddings'].tolist()
all_embeddings = np.vstack(filtered_embeddings)
dimension = all_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(all_embeddings)
faiss.write_index(index, "editais_faiss_index.bin")

#### Funções de busca de similaridade

In [ ]:
index = faiss.read_index("editais_faiss_index.bin")
print("Índice FAISS carregado com sucesso!")

Índice FAISS carregado com sucesso!


In [ ]:
def find_most_similar_chunks(query, top_k=1000):
    query_embedding = encoder.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = []
    # total_chunks = sum(len(chunks) for chunks in df_rag['text_chunks'])
    total_chunks = sum(len(chunks) if chunks is not None else 0 for chunks in df_rag['text_chunks'])
    for i, idx in enumerate(indices[0]):
        if idx < total_chunks:
            doc_idx = 0
            chunk_idx = idx
            while chunk_idx >= len(df_rag['text_chunks'].iloc[doc_idx]):
                chunk_idx -= len(df_rag['text_chunks'].iloc[doc_idx])
                doc_idx += 1
            results.append({
                'edital': df_rag['edital_number'].iloc[doc_idx],
                'document': df_rag['pdf_link'].iloc[doc_idx],
                'chunk': df_rag['text_chunks'].iloc[doc_idx][chunk_idx],
                'distance': distances[0][i]
            })
    return results

def query_documents(query):
    similar_chunks = find_most_similar_chunks(query)
    context = " ".join([result['chunk'].replace("\n", "") for result in similar_chunks])
    # response = generate_response(query, context)
    return similar_chunks

In [ ]:
most_similar = find_most_similar_chunks("O meu projeto é sobre Inteligência artificial, matchmaking de projetos com editais de fomento à inovação", top_k=100)
most_similar

[{'edital': 745,
  'document': 'http://finep.gov.br/images/chamadas-publicas/2024/07_08_2024_Regulamento_Conhecimento_Brasil.pdf',
  'chunk': 'A presente ação encontra-se prevista no âmbito dos Programas Estruturantes e \nMobilizadores do MCTI, vinculada ao Programa 5 - Repatriação de Talentos (Conhecimento Brasil), \nque busca a repatriação de talentos científicos, tecnológicos e inovadores a serem alocados em \nInstitutos de Ciência e Tecnologia (ICTs) e empresas brasileiras para o desenvolvimento de projetos \nfocados em programas estratégicos nacionais, no desenvolvimento industrial em áreas prioritárias e \nna redução de assimetrias do Sistema Nacional de Ciência, Tecnologia e Inovação. 1.3. Entende-se por risco tecnológico a possibilidade de insucesso no desenvolvimento de \nsolução, decorrente de processo em que o resultado é incerto em função do conhecimento técnico-\ncientífico insuficiente à época em que se decide pela realização da ação (art. 2°, inciso III, decreto \n9.283/

TODO: Pensar uma maneira de ordenar dependendo de quantas vezes aparecem, dando um peso pela distância

In [ ]:
from collections import defaultdict
def aggregate_similarities_by_edital(similar_chunks):
    edital_similarity = defaultdict(list)

    # Agrupar chunks por edital
    for result in similar_chunks:
        edital_similarity[result['edital']].append(result['distance'])

    # Calcular a média das distâncias para cada edital
    edital_avg_similarity = {}
    for edital, distances in edital_similarity.items():
        avg_distance = sum(distances) / len(distances)
        edital_avg_similarity[edital] = avg_distance

    # Ordenar os editais pela menor média de distância (maior similaridade)
    sorted_edital_similarity = sorted(edital_avg_similarity.items(), key=lambda x: x[1])

    return sorted_edital_similarity

In [ ]:
aggregate_similarities_by_edital(most_similar)

[(745, 0.8844807147979736),
 (598, 0.9292677938938141),
 (738, 0.936956524848938),
 (666, 0.9720920324325562),
 (614, 0.9776144027709961),
 (728, 0.9778879404067993),
 (649, 0.9812165945768356),
 (619, 0.9826295971870422),
 (739, 0.991142988204956),
 (715, 0.9925986528396606),
 (635, 0.996788740158081),
 (731, 0.9969049294789633),
 (735, 1.003000259399414),
 (730, 1.0107453671785502),
 (737, 1.017249345779419),
 (591, 1.0180174112319946),
 (675, 1.019078016281128),
 (726, 1.0192632675170898),
 (729, 1.0192632675170898),
 (732, 1.0192632675170898),
 (656, 1.0212082862854004),
 (733, 1.0216882705688477),
 (676, 1.0236806869506836),
 (705, 1.0240695476531982)]

In [ ]:
most_similar = find_most_similar_chunks("O meu projeto é sobre tratamento de água com menos poluentes", top_k=100)
aggregate_similarities_by_edital(most_similar)

[(698, 0.6188107579946518),
 (645, 0.6392405331134796),
 (718, 0.6496778130531311),
 (669, 0.6860682010650635),
 (741, 0.6882257461547852),
 (740, 0.6893746070563793),
 (682, 0.6926972270011902),
 (590, 0.6929644346237183),
 (676, 0.6934251189231873),
 (742, 0.696500403540475),
 (679, 0.7134519591927528),
 (690, 0.7169102907180787),
 (634, 0.7203289270401001),
 (618, 0.7213410139083862),
 (643, 0.7223404049873352),
 (691, 0.72636978328228),
 (656, 0.7308749556541443),
 (739, 0.7378624081611633),
 (653, 0.7409210205078125),
 (631, 0.74251788854599),
 (705, 0.7444129884243011),
 (630, 0.7461254596710205),
 (703, 0.7472071796655655),
 (681, 0.7516208291053772),
 (685, 0.7517645359039307),
 (654, 0.753885293006897)]

## Nome do edital

In [ ]:
import requests
from lxml import html
import pandas as pd

# Função para extrair o conteúdo com base no XPath
def importxml(page, xpath):
    tree = html.fromstring(page.content)
    content_list = tree.xpath(xpath)
    content = content_list[0].text_content() if len(content_list) > 0 else ''
    return content.strip()

# Função para extrair todos os links de PDF de um edital
def extract_name_from_edital(url):
    page = requests.get(url)

    name = html.fromstring(page.content).xpath('//*[@id="componente"]/div/h2/a')

    if len(name):
      print(name[0].text)
      name = name[0].text
    else:
      name = ''

    return name

# Lista para armazenar as informações
data = []

editais_numbers = list(range(586, 748))

# Loop por cada edital
for edital_number in editais_numbers:
    url = f'http://finep.gov.br/chamadas-publicas/chamadapublica/{edital_number}'
    name = extract_name_from_edital(url)

    # Adiciona cada PDF do edital à lista de dados

    data.append({'edital_number': edital_number, 'edital_name': name})

# Convertendo a lista em dataframe
df_names = pd.DataFrame(data)

# Salvar os dados em um arquivo CSV
# df.to_csv('editais_finep.csv', index=False)

print("Extração concluída e dados salvos em 'editais_finep.csv'")
df_names

Chamada PÃºblica Conjunta Finep - TEKES Projetos de P,D&I Industrial
Edital de SeleÃ§Ã£o PÃºblica MCTI/AEB/FINEP/FNDCT â SubvenÃ§Ã£o EconÃ´mica Ã  InovaÃ§Ã£o â TransferÃªncia de Tecnologia do SGDC â 01/2015
Edital MCTI/SECIS/FINEP/FNDCT Viver Sem Limite
Edital Finep/BNDES - Padiq
Chamada PÃºblica Finep e Conselho NorueguÃªs
Chamada PÃºblica MCTI/FINEP/FNDCT 01/2016 - Zika
Institutos de Pesquisa do MCTI
Centros Nacionais MultiusuÃ¡rios
Apoio Institucional ao Sistema Nacional de C,T&I
Plano de Desenvolvimento, Sustentabilidade e InovaÃ§Ã£o no Setor de MineraÃ§Ã£o e TransformaÃ§Ã£o Mineral â INOVA MINERAL
Finep / Vinnova - CooperaÃ§Ã£o ICT-Empresa - 08/2016
AMAZÃNIA LEGAL 04/2016
ERA-MIN 2 - 2017
Finep/CDTI - COOPERAÃÃO ICT-EMPRESA â 01/2017
Programa de Investimento em Startups Inovadoras
Finep e NRC (National Research Council of Canada)
Chamada PÃºblica Bilateral Finep â CDTI para Projetos de InovaÃ§Ã£o TecnolÃ³gica entre Empresas do Brasil e da Espanha
Investimento em Sta

,edital_number,edital_name
0,586,Chamada PÃºblica Conjunta Finep - TEKES Projet...
1,587,Edital de SeleÃ§Ã£o PÃºblica MCTI/AEB/FINEP/FN...
2,588,Edital MCTI/SECIS/FINEP/FNDCT Viver Sem Limite
3,589,
4,590,Edital Finep/BNDES - Padiq
...,...,...
157,743,Edital FIP Nordeste Capital Semente
158,744,RecuperaÃ§Ã£o e preservaÃ§Ã£o de acervos 2024
159,745,Conhecimento Brasil
160,746,Chamada pÃºblica de Planos de NegÃ³cios para i...


In [ ]:
df_names.to_csv('editais_finep_names.csv', index=False)
df_names.to_json('editais_finep_names.json', orient='records')